In [1]:
from dotenv import load_dotenv

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)
from api_helpers.config import Config
import pandas as pd

config = Config()

In [2]:
from api_helpers.clients import get_betfair_client, get_postgres_client

bf_client = get_betfair_client()
pg_client = get_postgres_client()

2025-09-15T10:36:17Z | INFO - Logging configuration initialized with level: INFO
2025-09-15T10:36:17Z | INFO - Logging into Betfair...
2025-09-15T10:36:17Z | INFO - Logged into Betfair!


In [3]:
# Run DB query and sync Betfair call concurrently
from datetime import datetime, timedelta


start = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
end = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

bf_client.get_past_orders_by_date_range(start, end)

2025-09-05T19:52:54Z | INFO - No cleared orders found


""


In [ ]:
current_orders = bf_client.get_current_orders()

In [ ]:
current_orders = current_orders[
    current_orders["execution_status"] == "EXECUTION_COMPLETE"
]

In [17]:
current_orders.groupby(["market_id", "selection_id", "selection_type"]).agg(
    {"size_matched": "sum", "average_price_matched": "mean"}
).reset_index().round(2)

,market_id,selection_id,selection_type,size_matched,average_price_matched
0,1.247398601,59587502,BACK,50.0,3.0


In [14]:
current_orders

,bet_id,market_id,selection_id,selection_type,execution_status,placed_date,matched_date,average_price_matched,customer_strategy_ref,size_matched,size_remaining,size_lapsed,size_cancelled,size_voided,price,size
0,400751391457,1.247398601,59587502,BACK,EXECUTION_COMPLETE,2025-09-05T18:30:01.000Z,2025-09-05T18:30:01.000Z,3.05,trader,26.0,0.0,0.0,0.0,0.0,3.05,26.0
1,400752233849,1.247398601,59587502,BACK,EXECUTION_COMPLETE,2025-09-05T18:36:56.000Z,2025-09-05T18:36:56.000Z,3.00,trader,9.0,0.0,0.0,0.0,0.0,3.00,9.0
2,400752338888,1.247398601,59587502,BACK,EXECUTION_COMPLETE,2025-09-05T18:37:48.000Z,2025-09-05T18:37:48.000Z,2.98,trader,5.0,0.0,0.0,0.0,0.0,2.98,5.0
3,400753030895,1.247398601,59587502,BACK,EXECUTION_COMPLETE,2025-09-05T18:42:57.000Z,2025-09-05T18:42:58.000Z,2.96,trader,10.0,0.0,0.0,0.0,0.0,2.96,10.0


In [4]:
bf = bf_client.create_market_data()

2025-09-15T10:36:48Z | INFO - Found 62 markets


In [7]:
bf[bf['course'] == 'Brighton']['race_time'].unique()

<DatetimeArray>
['2025-09-15 14:12:00+01:00', '2025-09-15 14:42:00+01:00',
 '2025-09-15 15:12:00+01:00', '2025-09-15 15:42:00+01:00',
 '2025-09-15 16:12:00+01:00', '2025-09-15 16:42:00+01:00',
 '2025-09-15 17:12:00+01:00']
Length: 7, dtype: datetime64[ns, Europe/London]

In [8]:
bf[bf['race_time']== '2025-09-15 16:12:00+01:00']

,race_time,market,race,course,horse,status,market_id,todays_betfair_selection_id,last_traded_price,total_matched,...,lay_price_3,lay_price_3_depth,lay_price_4,lay_price_4_depth,lay_price_5,lay_price_5_depth,total_matched_event,percent_back_win_book,percent_lay_win_book,market_width
292,2025-09-15 16:12:00+01:00,WIN,7f Hcap,Brighton,Tenbob Tommy,ACTIVE,1.247725155,76728350,3.65,1411.25,...,3.80,2.0,3.85,12.0,3.90,10.0,4062,103,98,5
293,2025-09-15 16:12:00+01:00,WIN,7f Hcap,Brighton,Kranjcar,ACTIVE,1.247725155,70867341,4.30,1347.77,...,4.60,2.0,4.70,7.0,4.80,9.0,4062,103,98,5
294,2025-09-15 16:12:00+01:00,WIN,7f Hcap,Brighton,Rockstar Icon,ACTIVE,1.247725155,56460442,6.20,662.83,...,6.60,7.0,6.80,8.0,7.00,7.0,4062,103,98,5
295,2025-09-15 16:12:00+01:00,WIN,7f Hcap,Brighton,Silent Flame,ACTIVE,1.247725155,38782427,6.20,262.46,...,6.80,7.0,7.00,13.0,7.20,7.0,4062,103,98,5
296,2025-09-15 16:12:00+01:00,WIN,7f Hcap,Brighton,Blue Collar Lad,ACTIVE,1.247725155,39050479,10.50,160.29,...,12.50,1.0,13.00,7.0,14.00,4.0,4062,103,98,5
297,2025-09-15 16:12:00+01:00,WIN,7f Hcap,Brighton,Marsh Benham,ACTIVE,1.247725155,39439919,12.50,217.01,...,16.00,8.0,16.50,6.0,17.00,8.0,4062,103,98,5
298,2025-09-15 16:12:00+01:00,PLACE,To Be Placed,Brighton,Tenbob Tommy,ACTIVE,1.247725157,76728350,2.52,4.05,...,2.26,3.0,2.28,20.0,1000.00,0.0,126,207,190,17
299,2025-09-15 16:12:00+01:00,PLACE,To Be Placed,Brighton,Kranjcar,ACTIVE,1.247725157,70867341,2.02,65.73,...,2.46,9.0,2.54,6.0,10.00,8.0,126,207,190,17
300,2025-09-15 16:12:00+01:00,PLACE,To Be Placed,Brighton,Rockstar Icon,ACTIVE,1.247725157,56460442,3.10,30.01,...,3.45,13.0,3.50,15.0,3.55,5.0,126,207,190,17
301,2025-09-15 16:12:00+01:00,PLACE,To Be Placed,Brighton,Silent Flame,ACTIVE,1.247725157,38782427,3.05,10.74,...,3.30,6.0,3.35,2.0,1000.00,0.0,126,207,190,17
